In [2]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from enum import Enum

In [7]:
load_dotenv()
OER_API_KEY = os.getenv("OER_API_KEY")

In [18]:
class Currency(Enum):
    GBP = "GBP"
    EUR = "EUR"
    USD = "USD"
    JPY = "JPY"
    CHF = "CHF"
    CAD = "CAD"
    AUD = "AUD"
    NZD = "NZD"
    ZAR = "ZAR"
    CNY = "CNY"
    HKD = "HKD"

class OpenExchangeRateApi:
    def __init__(self, api_key: str) -> None:
        self.api_key = api_key
        self.base_url = "https://openexchangerates.org/api/latest.json"
        self.base_currency = Currency.USD # TODO: not supported in free plan
        self.default_exchange_currencies = [
            Currency.GBP,
            Currency.EUR,
            Currency.USD,
            Currency.JPY, 
            Currency.CHF, 
            Currency.CAD, 
            Currency.AUD]

    def get_latest(self, base_currency: Currency = Currency.GBP, exchangee_currencies: list[Currency] = None ) -> dict:
        if exchangee_currencies is None:
            exchangee_currencies = self.default_exchange_currencies
        params = {
            "app_id": self.api_key,
            # "base": self.base_currency, # TODO: not supported in free plan
            "symbols": ",".join([currency.value for currency in exchangee_currencies])
        }
        response = requests.get(self.base_url, params=params)
        if response.status_code != 200:
            print(response.status_code)
            print(response.text)
            raise Exception("Failed to get jobs")
        return response.json()

def load(dict_list: dict) -> pd.DataFrame:
    timestamp = dict_list["timestamp"] # UTC timestamp indicating the time the data was collected
    pandas_timestamp = pd.Timestamp(timestamp, unit="s")
    base_currency = dict_list["base"]
    rates = dict_list["rates"]
    df = pd.DataFrame.from_dict(rates, orient="index", columns=["rate"])
    df["timestamp"] = pandas_timestamp
    df["base_currency"] = base_currency
    return df
    

In [8]:
oer = OpenExchangeRateApi(OER_API_KEY)
exchanges = oer.get_latest()
exchanges

{'disclaimer': 'Usage subject to terms: https://openexchangerates.org/terms',
 'license': 'https://openexchangerates.org/license',
 'timestamp': 1693306800,
 'base': 'USD',
 'rates': {'AUD': 1.554636,
  'CAD': 1.36143,
  'CHF': 0.885177,
  'EUR': 0.925348,
  'GBP': 0.794236,
  'JPY': 146.784875,
  'USD': 1}}

In [19]:
df = load(exchanges)
df

,rate,timestamp,base_currency
AUD,1.554636,2023-08-29 11:00:00,USD
CAD,1.361430,2023-08-29 11:00:00,USD
CHF,0.885177,2023-08-29 11:00:00,USD
EUR,0.925348,2023-08-29 11:00:00,USD
GBP,0.794236,2023-08-29 11:00:00,USD
JPY,146.784875,2023-08-29 11:00:00,USD
USD,1.000000,2023-08-29 11:00:00,USD
